# 差分 高速化比較


参考
- http://dr-kayai.hatenablog.com/entry/2014/10/09/175456
- https://kyotogeopython.zawawahoge.com/html/%E3%82%B9%E3%83%A9%E3%82%A4%E3%83%89/Python%E8%AC%9B%E7%BF%92%E4%BC%9A%EF%BC%88%E5%BF%9C%E7%94%A8%E7%B7%A8%EF%BC%89.html

In [258]:
from numba import njit, jit, f8
import numpy as np
import matplotlib.pyplot as plt
import time
%matplotlib inline

In [259]:
#zb チェック
@jit
def draw_v2d(x, y, z):
    import matplotlib.pyplot as plt
    plt.contourf(x,y,z)
    return 0

In [282]:
# node 
imax = 10000; jmax = 10000

#河床高
zb = np.random.rand(imax, jmax)

#mesh
x, y = np.meshgrid(np.arange(zb.shape[1]), np.arange(zb.shape[0]))

#res = draw_v2d(x, y, zb)

print(zb.shape)

(10000, 10000)


In [261]:
#ノーマル
def func_0(zb):
    
    dzdx = np.zeros(zb.shape)
    dzdy = np.zeros(zb.shape)
    
    for i in range(1,imax):
        for j in range(0,jmax):
            dzdx[i,j] = zb[i,j] - zb[i-1,j]
            
    for i in range(0,imax):
        for j in range(1,jmax):
            dzdy[i,j] = zb[i,j] - zb[i,j-1]
             
    return dzdx, dzdy

In [262]:
#スライス
def func_1(zb):
    
    dzdx = np.zeros(zb.shape)
    dzdy = np.zeros(zb.shape)
    
    dzdx[1:,0:] = zb[1:, 0:] - zb[0:-1,0:]
    dzdy[0:,1:] = zb[0:, 1:] - zb[0:,0:-1]

    return dzdx

In [263]:
#numba jit
@jit("f8[:,:](f8[:,:])", nopython=True)
def func_2(zb):

    dzdx = np.zeros(zb.shape)
    dzdy = np.zeros(zb.shape)
    
    dzdx[1:,0:] = zb[1:, 0:] - zb[0:-1,0:]
    dzdy[0:,1:] = zb[0:, 1:] - zb[0:,0:-1]
    
    return dzdx


In [264]:
#numba jit
@jit("f8[:,:](f8[:,:])", nopython=True)
def func_3(zb):
    
    
    dzdx = np.zeros(zb.shape)
    dzdy = np.zeros(zb.shape)
    
    imax = zb.shape[1]
    jmax = zb.shape[0]
    
    for i in range(1,imax):
        for j in range(0,jmax):
            dzdx[i,j] = zb[i,j] - zb[i-1,j]
            
    for i in range(0,imax):
        for j in range(1,jmax):
            dzdy[i,j] = zb[i,j] - zb[i,j-1]
    
    return dzdx


In [265]:
#numba jit
@jit("f8[:,:](f8[:,:])", nopython=True, parallel=True)
def func_4(zb):
    
    
    dzdx = np.zeros(zb.shape)
    dzdy = np.zeros(zb.shape)
    
    imax = zb.shape[1]
    jmax = zb.shape[0]
    
    for i in range(1,imax):
        for j in range(0,jmax):
            dzdx[i,j] = zb[i,j] - zb[i-1,j]
            
    for i in range(0,imax):
        for j in range(1,jmax):
            dzdy[i,j] = zb[i,j] - zb[i,j-1]
    
    return dzdx

In [266]:
#スライス+parallel
@jit("f8[:,:](f8[:,:])", nopython=True, parallel=True)
def func_5(zb):
    
    dzdx = np.zeros(zb.shape)
    dzdy = np.zeros(zb.shape)
    
    dzdx[1:,0:] = zb[1:, 0:] - zb[0:-1,0:]
    dzdy[0:,1:] = zb[0:, 1:] - zb[0:,0:-1]

    return dzdx

In [267]:
print("for")
%timeit func_0(zb)
print("slice")
%timeit func_1(zb)
print("jit+slice")
%timeit func_2(zb)

for
1.03 s ± 13.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
slice
15 ms ± 168 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
jit+slice
16.5 ms ± 312 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [268]:
print("jit+for")
%timeit func_3(zb)
print("jit+for+parallel")
%timeit func_4(zb)
print("jit+slice+parallel")
%timeit func_5(zb)

jit+for
8.27 ms ± 77.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
jit+for+parallel
5.65 ms ± 177 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
jit+slice+parallel
3.3 ms ± 109 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [271]:
#時間計測
print("for")
t0 = time.time()  
print("start time : ", t0)        
res = func_0(zb)
t1 = time.time()
print("End time : ", t1)
print("Elapsed : ", t1 - t0)

for
start time :  1591689694.7566092
End time :  1591689695.820787
Elapsed :  1.0641777515411377


In [270]:
print("slice")
t0 = time.time()  
print("start time : ", t0)        
res = func_1(zb)
t1 = time.time()
print("End time : ", t1)
print("Elapsed : ", t1 - t0)

slice
start time :  1591689693.0761821
End time :  1591689693.0941298
Elapsed :  0.017947673797607422


In [279]:
print("jit+slice")
t0 = time.time()  
print("start time : ", t0)        
res = func_2(zb)
t1 = time.time()
print("End time : ", t1)
print("Elapsed : ", t1 - t0)

jit+slice
start time :  1591690272.7082362
End time :  1591690272.9226604
Elapsed :  0.21442413330078125


In [273]:
print("jit+for")
t0 = time.time()  
print("start time : ", t0)        
res = func_3(zb)
t1 = time.time()
print("End time : ", t1)
print("Elapsed : ", t1 - t0)

jit+for
start time :  1591689714.1537569
End time :  1591689714.1657243
Elapsed :  0.01196742057800293


In [278]:
print("jit+for+parallel")
t0 = time.time()  
print("start time : ", t0)        
res = func_4(zb)
t1 = time.time()
print("End time : ", t1)
print("Elapsed : ", t1 - t0)

jit+for+parallel
start time :  1591690266.8758657
End time :  1591690266.954654
Elapsed :  0.07878828048706055


In [283]:
print("jit+slice+parallel")
t0 = time.time()  
print("start time : ", t0)        
res = func_5(zb)
t1 = time.time()
print("End time : ", t1)
print("Elapsed : ", t1 - t0)

jit+slice+parallel
start time :  1591690418.1483846
End time :  1591690418.3548305
Elapsed :  0.20644593238830566
